#### Importing libraries

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import nltk
from nltk.corpus import stopwords
import string
import re
from fuzzywuzzy import process
import fuzzymatcher
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#### Reading files

In [19]:
## Readiing files
df1 = pd.read_csv('source_1.csv')
df2 = pd.read_csv('source_2.csv')

In [20]:
df1.head()

,id,name
0,0,"Horses, asses, mules and hinnies; live, pure-b..."
1,1,"Horses; live, pure-bred breeding animals"
2,2,"Horses; live, other than pure-bred breeding an..."
3,4,"Horses, asses, mules and hinnies; live, other ..."
4,5,"Bovine animals; live, pure-bred breeding animals"


In [21]:
df2.head()

,id,name
0,0,leveillula lactucae-serriolae
1,1,podosphaera aphanis
2,2,lathyrus czeczottianus
3,3,crocus biflorus subsp. caricus
4,4,hordeum brevisubulatum


In [22]:
print(df1['id'].duplicated().sum())

947


In [23]:
pd.concat(g for _, g in df1.groupby("id") if len(g) > 1)

,id,name
0,0,"Horses, asses, mules and hinnies; live, pure-b..."
1580,0,"Wood in the rough, even peeled, or roughly squ..."
1,1,"Horses; live, pure-bred breeding animals"
1581,1,"Wood in the rough, even peeled, or roughly squ..."
2,2,"Horses; live, other than pure-bred breeding an..."
...,...,...
2766,2104,"Peaches, otherwise prepared or preserved"
1574,2107,"Parts of lawn mowers f/harvesting/threshing, etc"
2767,2107,"Mixtures of fruits, prepared or preserved"
1575,2108,Machinery for processing milk


In [24]:
df1['id'] = df1.index

In [25]:
df2['id'] = df2.index

In [26]:
df1['id'].duplicated().sum()

0

#### Data preprocessing

In [27]:
# code to count number of duplicates in df1 and df2
print("duplicates in df1 are:", df1['name'].duplicated().sum())
print("duplicates in df2 are:", df2['name'].duplicated().sum())

duplicates in df1 are: 656
duplicates in df2 are: 69


In [28]:
# remove duplicates from df1 and df2
# to avoid irrelevant redundancy
df1_unique = df1.drop_duplicates(subset = "name", keep='first')
df2_unique = df2.drop_duplicates(subset = "name", keep = 'first')

In [29]:
df1_unique.shape

(12582, 2)

In [30]:
df1.shape

(13238, 2)

In [31]:
# remove punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
df1_unique['name_nonpunct'] = df1_unique['name'].apply(lambda x: remove_punct(x))
df1_unique.head()

,id,name,name_nonpunct
0,0,"Horses, asses, mules and hinnies; live, pure-b...",Horses asses mules and hinnies live purebred b...
1,1,"Horses; live, pure-bred breeding animals",Horses live purebred breeding animals
2,2,"Horses; live, other than pure-bred breeding an...",Horses live other than purebred breeding animals
3,3,"Horses, asses, mules and hinnies; live, other ...",Horses asses mules and hinnies live other than...
4,4,"Bovine animals; live, pure-bred breeding animals",Bovine animals live purebred breeding animals


In [32]:
# getting stop words
stopword = nltk.corpus.stopwords.words('english')

In [33]:
# Tokenize words
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens
df1_unique['tokenized'] = df1_unique['name_nonpunct'].apply(lambda x: tokenize(x.lower()))
df1_unique.head()

,id,name,name_nonpunct,tokenized
0,0,"Horses, asses, mules and hinnies; live, pure-b...",Horses asses mules and hinnies live purebred b...,"[horses, asses, mules, and, hinnies, live, pur..."
1,1,"Horses; live, pure-bred breeding animals",Horses live purebred breeding animals,"[horses, live, purebred, breeding, animals]"
2,2,"Horses; live, other than pure-bred breeding an...",Horses live other than purebred breeding animals,"[horses, live, other, than, purebred, breeding..."
3,3,"Horses, asses, mules and hinnies; live, other ...",Horses asses mules and hinnies live other than...,"[horses, asses, mules, and, hinnies, live, oth..."
4,4,"Bovine animals; live, pure-bred breeding animals",Bovine animals live purebred breeding animals,"[bovine, animals, live, purebred, breeding, an..."


In [34]:
# remove stop words
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    text2 = " ".join(text)
    return text2
df1_unique['text_nonstop'] = df1_unique['tokenized'].apply(lambda x: remove_stopwords(x))
df1_unique.head()

,id,name,name_nonpunct,tokenized,text_nonstop
0,0,"Horses, asses, mules and hinnies; live, pure-b...",Horses asses mules and hinnies live purebred b...,"[horses, asses, mules, and, hinnies, live, pur...",horses asses mules hinnies live purebred breed...
1,1,"Horses; live, pure-bred breeding animals",Horses live purebred breeding animals,"[horses, live, purebred, breeding, animals]",horses live purebred breeding animals
2,2,"Horses; live, other than pure-bred breeding an...",Horses live other than purebred breeding animals,"[horses, live, other, than, purebred, breeding...",horses live purebred breeding animals
3,3,"Horses, asses, mules and hinnies; live, other ...",Horses asses mules and hinnies live other than...,"[horses, asses, mules, and, hinnies, live, oth...",horses asses mules hinnies live purebred breed...
4,4,"Bovine animals; live, pure-bred breeding animals",Bovine animals live purebred breeding animals,"[bovine, animals, live, purebred, breeding, an...",bovine animals live purebred breeding animals


#### Matching words with fuzzymatcher

In [35]:
# matching similar words
matched = fuzzymatcher.fuzzy_left_join(df1_unique, df2_unique, left_on = "text_nonstop", right_on = "name", 
                             left_id_col="id",
                             right_id_col="id")

In [36]:
matched.head(3)

,best_match_score,__id_left,__id_right,id_left,name_left,name_nonpunct,tokenized,text_nonstop,id_right,name_right
0,-0.542132,0,29307.0,0,"Horses, asses, mules and hinnies; live, pure-b...",Horses asses mules and hinnies live purebred b...,"[horses, asses, mules, and, hinnies, live, pur...",horses asses mules hinnies live purebred breed...,29307.0,live mulches
81,-0.144357,1,27426.0,1,"Horses; live, pure-bred breeding animals",Horses live purebred breeding animals,"[horses, live, purebred, breeding, animals]",horses live purebred breeding animals,27426.0,animal breeding
160,-0.144357,2,27426.0,2,"Horses; live, other than pure-bred breeding an...",Horses live other than purebred breeding animals,"[horses, live, other, than, purebred, breeding...",horses live purebred breeding animals,27426.0,animal breeding


In [37]:
# sorting best_match_score by descending 
matched3 = matched.sort_values('best_match_score', ascending=False).groupby(['name_left','name_right'], sort=False).first().reset_index()

In [38]:
matched3.head(3)

,name_left,name_right,best_match_score,__id_left,__id_right,id_left,name_nonpunct,tokenized,text_nonstop,id_right
0,Magnetic resonance imaging apparatus,magnetic resonance imaging,1.014215,11791,21243.0,11791,Magnetic resonance imaging apparatus,"[magnetic, resonance, imaging, apparatus]",magnetic resonance imaging apparatus,21243.0
1,Sewage sludge,sewage sludge,0.827112,7007,41053.0,7007,Sewage sludge,"[sewage, sludge]",sewage sludge,41053.0
2,Hydrogen peroxide,hydrogen peroxide,0.730148,3867,21739.0,3867,Hydrogen peroxide,"[hydrogen, peroxide]",hydrogen peroxide,21739.0


In [39]:
# columns = ['best_match_score', '__id_left', '__id_right', 'id_left', 'name_left',
#        'name_nonpunct', 'tokenized', 'text_nonstop', 'id_right', 'name_right']

In [40]:
# sorted_values = matched[columns].sort_values(by=['best_match_score'], ascending=False)

In [41]:
# sorted_values.dropna(inplace = True)

In [42]:
# sorted_values.tail(5)

In [43]:
# sorted2 = sorted_values.drop_duplicates(subset = ['name_left'], keep='first')

In [57]:
matched3 = matched3.drop_duplicates(subset = ['name_right'], keep='first')

In [58]:
# sorted3.head(3)

In [59]:
# convert id to int
# sorted3['id_right'] = sorted3['id_right'].astype(int)

In [60]:
# sorted3['id_right'] = sorted3['id_right'].astype(int)

In [61]:
# sorted3.info()

In [71]:
print(matched3['id_right'].duplicated().sum())

0


In [63]:
matched3.shape

(2399, 10)

In [64]:
# convert id from float to int
matched3['id_right'] = matched3['id_right'].astype(int)

# printing dataframe to csv
predicted = matched3[['id_left', 'id_right']]
# predicted.sort_values(by = ['id_left'], inplace = True)
predicted = predicted.sort_values('id_left', ascending=True).groupby(['id_left','id_right'], sort=False).first().reset_index()

# rename columns
predicted.rename(columns = {'id_left':'source1', 'id_right':'source2'}, inplace = True)

# create csv file
predicted.to_csv('benefatia@gmail.com.csv', index = False)

In [65]:
predicted

,source1,source2
0,3,29307
1,23,3889
2,24,39686
3,25,39057
4,26,37492
...,...,...
2394,13212,46327
2395,13216,8834
2396,13223,11391
2397,13230,44137


In [66]:
predicted.shape

(2399, 2)

In [67]:
# visualizaing the matched data from a dataframe
matched_data = matched3[['name_left', 'name_right']]
# rename columns
matched_data.rename(columns = {'name_left':'source1', 'name_right':'source2'}, inplace = True)
matched_data

,source1,source2
0,Magnetic resonance imaging apparatus,magnetic resonance imaging
1,Sewage sludge,sewage sludge
2,Hydrogen peroxide,hydrogen peroxide
3,Magnesium phosphide,magnesium phosphide
4,Milking machines,milking machines
...,...,...
10611,"Coconut, abaca (Manila hemp or Musa textilis N...",musa textilis
10615,Food preparations; bakers' wares n.e.s. in hea...,pharmaceutical products
10617,"Amêijoas, berbigões and arcas (families Arcida...",arcticidae
10623,"Veículos de chassis articulado, p/ o transport...",p (symbol)


In [68]:
matched_data

,source1,source2
0,Magnetic resonance imaging apparatus,magnetic resonance imaging
1,Sewage sludge,sewage sludge
2,Hydrogen peroxide,hydrogen peroxide
3,Magnesium phosphide,magnesium phosphide
4,Milking machines,milking machines
...,...,...
10611,"Coconut, abaca (Manila hemp or Musa textilis N...",musa textilis
10615,Food preparations; bakers' wares n.e.s. in hea...,pharmaceutical products
10617,"Amêijoas, berbigões and arcas (families Arcida...",arcticidae
10623,"Veículos de chassis articulado, p/ o transport...",p (symbol)


In [60]:
# printing specific rows
# print(source1.loc[[104]])
# print(source2.loc[[204]])